# Install ffmpeg-python
for recorded audio decoding

In [ ]:
!pip install -q ffmpeg-python

# Installing transformers

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.1MB 6.8MB/s 
     |████████████████████████████████| 901kB 35.8MB/s 
     |████████████████████████████████| 3.3MB 43.1MB/s 


# Installing ctcdecode
Implementation of beam search algorithm for python. This is used to rescore output symbols from model using language model (kenlm in this case)

kenlm is a n-gram language model small and efficient.
We can use neural network based networks but it will be more resource intensive so kenlm is best choice to use as language model and is already used in many projects.

By default ctcdecode use kenlm for rescoring so I do not bother to setup the model.
- [ctcdecode repo](https://github.com/parlance/ctcdecode)
- [kenlm](https://github.com/kpu/kenlm)

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1063, done.
remote: Total 1063 (delta 0), reused 0 (delta 0), pack-reused 1063
Receiving objects: 100% (1063/1063), 759.71 KiB | 11.17 MiB/s, done.
Resolving deltas: 100% (513/513), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 13792, done.        
remote: Counting objects: 100% (105/105), done.        
remote: Compressing objects: 100% (58/58), done.        
remote: Total 13792 (delta 59), reused 74 (delta 34), pack-reused 13687        
Receiving objects: 100% (13792/13792), 5.48 MiB |

# Loading Dependencies
- pytorch
- Transformers library
- numpy
- ctcdecode (ctc beam search decoder with kenlm as language model)
- librosa

In [ ]:
import torch
import transformers
import numpy as np
import ctcdecode
import librosa

# Instantiate pretrained models
- Tokenizer
- Wav2Vec2 Model

The model takes as input a speech signal in any language (currently english because it was trained on english dataset) in its raw form. This audio data is one-dimensional and is passed to a multi-layer 1-d Convolutional neural network to generate audio representations of 25ms each

In [ ]:
tokenizer = transformers.Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = transformers.Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:358: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.eval()

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

# Recording and loading audio in colab

Taken from [ricardodeazambuja.com](https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/)

In [ ]:
# https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio(sr):
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  speech, rate = librosa.load(io.BytesIO(riff),sr=16000)
  return speech, sr

In [ ]:
#record or load any audio file of your choice here
speech, rate = get_audio(sr=16000)

# wave2vec2 vocabulary

[found here](https://huggingface.co/facebook/wav2vec2-base-960h/resolve/main/vocab.json)

In [ ]:
label_dict = {"<pad>": 0, 
              "<s>": 1, 
              "</s>": 2,
              "<unk>": 3,
              "|": 4,
              "E": 5,
              "T": 6,
              "A": 7,
              "O": 8,
              "N": 9,
              "I": 10,
              "H": 11,
              "S": 12,
              "R": 13,
              "D": 14,
              "L": 15,
              "U": 16,
              "M": 17,
              "W": 18,
              "C": 19,
              "F": 20,
              "G": 21,
              "Y": 22,
              "P": 23,
              "B": 24,
              "V": 25,
              "K": 26,
              "'": 27,
              "X": 28,
              "J": 29,
              "Q": 30,
              "Z": 31
              }
labels = [key for key, value in label_dict.items()]

# CTC Beam search
CTC Beam search algorithm combined with language model for rescoring probabilities output from language model.

This class handles all the things we just need to pass our models softmax output into this class object to decode

In [ ]:
class CTCBeamDecoder:
    def __init__(self, labels, blank_id=0, beam_size=100, kenlm_path=None):
        print("loading beam search with kenlm...")
        self.labels = labels

        # model_path = is the path to your external kenlm language model(LM). Default is none.
        # alpha = Weighting associated with the LMs probabilities. A weight of 0 means the LM has no effect.
        # beta = Weight associated with the number of words within our beam.

        self.ctcdecoder = ctcdecode.CTCBeamDecoder(
            self.labels, model_path=kenlm_path,
            alpha=0.6, beta=1,
            beam_width=beam_size, blank_id=blank_id)
        print("loading finished")

    def __call__(self, output, num_sentences=1):
        sentences = []
        for num in range(num_sentences):
          beam_result, beam_scores, timesteps, out_seq_len = self.ctcdecoder.decode(output)
          # beam_result[0][0][:out_seq_len[0][0]] get the top beam for the first item in batch
          sentences.append(self.output(beam_result[0][num], self.labels, out_seq_len[0][num]))
        return sentences

    def output(self, tokens, vocab, seq_len):
        out = ''.join([vocab[x] for x in tokens[0:seq_len]])
        # wave2vec implementation use | for space in vocabulary
        return out.replace("|", " ")

In [ ]:
# blank_id = ctc blank token (epsilon) which is <pad> in wave2vec vocabulary
decode_and_rescore = CTCBeamDecoder(kenlm_path=None, 
                                          labels=labels, 
                                          blank_id=label_dict.get("<pad>"), 
                                          beam_size=100)

loading beam search with kenlm...
loading finished


# Inferencing
- tokenizing(encoding) speech data and return pytorch tensor
- pass encodings to model
- converting model outputs into probabilities using softmax

In [ ]:
input_values = tokenizer(speech, return_tensors = 'pt').input_values
#logits (non-normalized predictions)
logits = model(input_values).logits
out_proba = torch.nn.functional.softmax(logits, dim=-1)

In [ ]:
predicted_ids = torch.argmax(out_proba, dim =-1)
results_ = tokenizer.decode(predicted_ids[0])

In [ ]:
print("Without Language Model")
print(results_)

Without Language Model
THE BOOK IS ON THE TABLE


# Applying rescoring algorithm 
using language model and beam search

In [ ]:
results = decode_and_rescore(out_proba, num_sentences=5)

In [ ]:
print("With Language Model Kenlm")
for result in results:
  print(result)

With Language Model Kenlm
THE BOOK IS ON THE TABLE 
ETHE BOOK IS ON THE TABLE 
THE BOOK IS ON THEI TABLE 
 THE BOOK IS ON THE TABLE 
THE BOOK IS ON THE TABLEE 


# Load Audio and transcribe

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/c/c8/Example.ogg -O example.ogg

--2021-04-29 12:20:09--  https://upload.wikimedia.org/wikipedia/commons/c/c8/Example.ogg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105243 (103K) [application/ogg]
Saving to: ‘example.ogg’

example.ogg         100%[===================>] 102.78K  --.-KB/s    in 0.03s   

2021-04-29 12:20:09 (2.98 MB/s) - ‘example.ogg’ saved [105243/105243]



### Audio Loading Functions

In [ ]:
from scipy.signal import resample
import numpy as np
import soundfile as sf

class AudioReader:
    def __init__(self, audio_path, sr=16000, dtype="float32"):
        self._sr = sr
        self._dtype = dtype
        self._audio_path = audio_path
    
    def read(self):
        data, sr = sf.read(self._audio_path, dtype=self._dtype)
        data = self.__resample_file(data, sr, self._sr)
        return data, self._sr

    def __resample_file(self, array, original_sr, target_sr):
        return resample(array, num=int(len(array)*target_sr/original_sr))

class AudioStreaming:
    def __init__(self, audio_path, blocksize, sr=16000, overlap=0, padding=None, dtype="float32"):
        assert blocksize >= 0, "blocksize cannot be 0 or negative"
        self._sr = sr
        self._orig_sr = sf.info(audio_path).samplerate
        self._sf_blocks = sf.blocks(audio_path,
                        blocksize=blocksize, 
                        overlap=overlap,
                        fill_value=padding,
                        dtype=dtype)

    def generator(self):
        for block in self._sf_blocks:
            chunk = self.__resample_file(block, self._orig_sr, self._sr)
            yield chunk, self._orig_sr

    def __resample_file(self, array, original_sr, target_sr):
        return resample(array, num=int(len(array)*target_sr/original_sr))

### Loading Audio one pass

load full audio at once and transcribe

In [ ]:
audio_reader = AudioReader("/content/example.ogg", sr=16000)

In [ ]:
block, sr = audio_reader.read()
print(sr)
print(block.shape)

16000
(97664, 2)


In [ ]:
input_values = tokenizer(block[:,0], return_tensors = 'pt').input_values
#logits (non-normalized predictions)
logits = model(input_values).logits
out_proba = torch.nn.functional.softmax(logits, dim=-1)

In [ ]:
predicted_ids = torch.argmax(out_proba, dim =-1)
results_ = tokenizer.decode(predicted_ids[0])

In [ ]:
print("Without Language Model")
print(results_)

Without Language Model
THIS IS AN EXAMPLE SOUND FILE IN AG FORBUS FORMA FROM WICIPAEDIA THE FREE ENCYCLOPAEDIA


In [ ]:
results = decode_and_rescore(out_proba, num_sentences=5)

In [ ]:
print("With Language Model Kenlm")
for result in results:
  print(result)

With Language Model Kenlm
THIS IS AN EXAMPLE SOUND FILE IN AG FORBUS FORMA FROM WICIPAEDIA THE FREE ENCYCLOPAEDIA 
THIS IS AN EXAMPLE SOUND FILE IN AUG FORBUS FORMA FROM WICIPAEDIA THE FREE ENCYCLOPAEDIA 
THIS IS AN EXAMPLE SOUND FILE IN AG FORBUS FORMA FROM WICHIPAEDIA THE FREE ENCYCLOPAEDIA 
THIS IS AN EXAMPLE SOUND FILE IN AUG FORBUS FORMA FROM WICHIPAEDIA THE FREE ENCYCLOPAEDIA 
THIS IS AN EXAMPLE SOUND FILE IN OG FORBUS FORMA FROM WICIPAEDIA THE FREE ENCYCLOPAEDIA 


### Splitting audio and load

first split audio into multiple blocks and pass each block for transcribing

In [ ]:
## splitting 5 sec
audio_stream = AudioStreaming(audio_path="/content/example.ogg", blocksize=16000*5, padding=0)

In [ ]:
for block, sr in audio_stream.generator():
  inputs = tokenizer(block[:,0], return_tensors='pt').input_values
  logits = model(inputs).logits
  predicted_ids = torch.argmax(logits, dim =-1)
  print(tokenizer.decode(predicted_ids[0]), end="")

THIS IS AN EXAMPLE SOUNDILEIN AG VORBUS FORMAWICKIPEDIA THE FREEPADIA

### transcribe block and decode a last

In [ ]:
## splitting 5 sec
audio_stream = AudioStreaming(audio_path="/content/example.ogg", blocksize=16000*5, padding=0)

In [ ]:
ctc_outs = torch.Tensor()

In [ ]:
for block, sr in audio_stream.generator():
  inputs = tokenizer(block[:,0], return_tensors='pt').input_values
  logits = model(inputs).logits
  logits = torch.nn.functional.softmax(logits, dim=-1)
  ctc_outs = torch.cat((ctc_outs, logits), dim=1)

In [ ]:
results = decode_and_rescore(ctc_outs, num_sentences=5)

In [ ]:
for result in results:
  print(result)

THIS IS AN EXAMPLE SOUNDILE IN AG VORBUS FORMA WICKIPEDIA THE FREE  PADIA 
THIS IS AN EXAMPLE SOUNDILE IN AG VORBUS FORMA WICKIPEDIA THE FREE PADIA 
THIS IS AN EXAMPLE SOUNDILE IN AG VORBUS FORMA WICKIPEDIA THE FREE IPADIA 
THIS IS AN EXAMPLE SOUNDILE IN AG VORBUS FORMA WICKIPEDIA THE FREE  IPADIA 
THIS IS AN EXAMPLE SOUNDILE IN AG VORBUS FORMA WICKIPETIA THE FREE  PADIA 
